In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import gc
import h5py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
def moodNamePrintFromLabel(n):
  if n == 0: result = 'angry '
  elif n == 1: result = 'disgust '
  elif n == 2: result = 'fear'
  elif n == 3: result = 'happy'
  elif n == 4: result = 'sad'
  elif n == 5: result = 'surprise'
  elif n == 6: result = 'neutral'
  return result

In [3]:

def load_data(hdf5_file, dataset_name):
    with h5py.File(hdf5_file, 'r') as hf:
        data = np.array(hf[dataset_name])
        labels = np.array(hf['combined_labels'])
    return data, labels


# Load data
data, labels = load_data('output files/combined_images_labels.h5', 'combined_images')

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    data, 
    labels, 
    test_size=0.2, 
    stratify=labels
)

In [4]:
def train_data_generator(images, labels, batch_size=64, target_size=(224, 224)):
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True
    )
    
    def generator():
        while True:
            for start in range(0, len(images), batch_size):
                end = min(start + batch_size, len(images))
                batch_images = images[start:end]
                batch_labels = labels[start:end]
                # Resize images in batch
                batch_images_resized = np.array([cv2.resize(img, target_size) for img in batch_images])
                # Convert grayscale images to RGB if necessary
                if batch_images_resized.shape[-1] == 1:
                    batch_images_resized = np.repeat(batch_images_resized, 3, axis=-1)
                # Data augmentation
                augmented_images = datagen.flow(batch_images_resized, batch_size=batch_size, shuffle=False)
                # Yield both images and labels
                for aug_images in augmented_images:
                    yield aug_images, batch_labels
                    break  # Ensure we only yield one batch from the generator
    return generator()




def val_data_generator(images, labels, batch_size=64, target_size=(224, 224)):
    def generator():
        while True:
            for start in range(0, len(images), batch_size):
                end = min(start + batch_size, len(images))
                batch_images = images[start:end]
                batch_labels = labels[start:end]
                # Resize images in batch
                batch_images_resized = np.array([cv2.resize(img, target_size) for img in batch_images])
                # Convert grayscale images to RGB if necessary
                if batch_images_resized.shape[-1] == 1:
                    batch_images_resized = np.repeat(batch_images_resized, 3, axis=-1)
                yield batch_images_resized, batch_labels
    return generator()


# Create data generators
train_gen = train_data_generator(X_train, y_train, batch_size=64)
validation_gen = val_data_generator(X_val, y_val, batch_size=64)

In [52]:
# train_gen = train_data_generator(X_train, y_train, batch_size=64)
# x_batch, y_batch = next(train_gen)

# # x_batch will be a numpy array with augmented images, and y_batch will be labels
# print(f"x_batch shape: {x_batch[0].shape}")  # Should be (64, 224, 224, 3) if batch_size=64
# print(f"y_batch shape: {y_batch.shape}")  

In [5]:
# Print the shapes of training and validation data
print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Validation labels shape: {y_val.shape}")


Training data shape: (32784, 224, 224, 3)
Training labels shape: (32784,)
Validation data shape: (8196, 224, 224, 3)
Validation labels shape: (8196,)


In [6]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers[:2]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=['sparse_categorical_accuracy'])

In [7]:

# Train the model
model.fit(train_gen, steps_per_epoch=1, epochs=1, validation_data=validation_gen, validation_steps=128)
# Save the model in Keras format
model.save('output files/model.keras')

1/1 ━━━━━━━━━━━━━━━━━━━━ 113s 113s/step - loss: 2.6344 - sparse_categorical_accuracy: 0.2031 - val_loss: 2.1717 - val_sparse_categorical_accuracy: 0.1339


In [8]:
# Save the model
try:
    model.save('output files/model.keras')
    print("Model saved successfully.")
except Exception as e:
    print(f"An error occurred while saving the model: {e}")

# Load the model
try:
    loaded_model = tf.keras.models.load_model('output files/model.keras')
    loaded_model.summary()
    print("Model loaded successfully.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")


Model saved successfully.
An error occurred while loading the model: Layer "dense" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_483>, <KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_484>]


In [27]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,222,039 (27.55 MB)

 Trainable params: 2,395,687 (9.14 MB)

 Non-trainable params: 34,976 (136.62 KB)

 Optimizer params: 4,791,376 (18.28 MB)

In [10]:

loaded_model = tf.keras.models.load_model('output files/model.keras')


ValueError: Layer 'dense' expected 1 input(s). Received 2 instead.

In [ ]:
score = loaded_model.evaluate(X_test, y_test, verbose=1)

In [ ]:
import urllib.request

# URL to download the Haar cascade XML file
url = 'https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml'

# Path to save the downloaded XML file
xml_file_path = 'haarcascade_frontalface_default.xml'

# Download the XML file
urllib.request.urlretrieve(url, xml_file_path)

print("Haar cascade XML file downloaded successfully.")


Haar cascade XML file downloaded successfully.


In [46]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt


face_casecade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


def videoToMoodDetection(video_path):
  #video_cap = cv2.VideoCapture(0)
  video_cap = cv2.VideoCapture(video_path)

  while (video_cap.isOpened()):
    ret, frame = video_cap.read()

    Gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_casecade.detectMultiScale(Gray_img, 1.3, 5)

    for (x, y, w, h) in faces:
      single_face = Gray_img[y:y+h, x:x+w]  # Extract face region

      # Convert single_face to RGB if it is grayscale
      if len(single_face.shape) == 2:  # Check if the image is grayscale
        single_face = cv2.cvtColor(single_face, cv2.COLOR_GRAY2RGB)

      resized_img = cv2.resize(single_face, (224, 224), interpolation=cv2.INTER_AREA)
      #printing the 48*48 pixel images which will be passed to the model
      plt.imshow(resized_img, cmap='gray')
      plt.show()

      #resize for passing to the model
      resized_img = np.reshape(resized_img, (1, 224, 224, 3)) / 255.0

      #passing to model
      result = np.argmax(loaded_model.predict(resized_img), axis=-1)
      if result is not None:
        print(moodNamePrintFromLabel(result))



def imageToMoodDetection(img_path):

  img = cv2.imread(img_path)

  Gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  faces = face_casecade.detectMultiScale(Gray_img, 1.3, 3)

  for (x, y, w, h) in faces:
    single_face = Gray_img[y:y+h, x:x+w]  # Extract face region

    # Convert single_face to RGB if it is grayscale
    if len(single_face.shape) == 2:  # Check if the image is grayscale
      single_face = cv2.cvtColor(single_face, cv2.COLOR_GRAY2RGB)

    resized_img = cv2.resize(single_face, (224, 224), interpolation=cv2.INTER_AREA)
    plt.imshow(resized_img)
    plt.show()      
    print(resized_img.shape)
    resized_img = np.reshape(resized_img, (1, 224, 224, 3)) / 255.0

      #passing to model
    result = np.argmax(loaded_model.predict(resized_img), axis=-1)
    if result is not None:
      print(moodNamePrintFromLabel(result))



In [15]:
import cv2

# Load the image
imageToMoodDetection('Test image/Testing1.jpg')
imageToMoodDetection('Test image/Testing2.jpg')
imageToMoodDetection('Test image/Testing3.jpg')
imageToMoodDetection('Test image/Testing4.jpg')
imageToMoodDetection('Test image/Testing5.webp')
imageToMoodDetection('Test image/Testing6.jpg')
imageToMoodDetection('Test image/Testing7.jpg')







NameError: name 'imageToMoodDetection' is not defined

In [ ]:

# videoToMoodDetection('/content/video.mp4')


In [ ]:
import matplotlib.pyplot as plt

single_image = np.array(x_test[13]*255)
print(single_image.shape)
single_image = np.reshape(single_image,(48,48),order = 'F')
print(single_image.shape)
plt.imshow(single_image, cmap='gray', vmin=0, vmax=255)
# s_img = np.array(single_image, shape=(48,48))

In [54]:
import tensorflow as tf
print(tf.__version__)  # Should be 2.17.0

import keras
print(keras.__version__)  # Should be 3.4.1


2.17.0
3.4.1
